In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, IntSlider, FloatSlider, Checkbox

def diffraction_playground(detector_x=100, num_slits=2, slit_width=3, slit_sep=20, wavelength=5.0, random_spacing=False,):
    size = 500
    detector_height = 50  # height of the 2D detector display
    
    # Slit positions along left edge
    mid = size // 2
    if random_spacing:
        slit_positions = np.random.randint(slit_width//2, size - slit_width//2, size=num_slits).tolist()
    else:
        if num_slits == 1:
            slit_positions = [mid]
        else:
            slit_positions = [(mid - (num_slits-1)*slit_sep//2 + i*slit_sep) for i in range(num_slits)]
    
    # 2D grid
    y = np.arange(size).reshape(-1,1)
    x = np.arange(size).reshape(1,-1)
    
    # Precompute distances from slits
    r_list = []
    for pos in slit_positions:
        r = np.sqrt((y - pos)**2 + x**2) + 1e-6
        r_list.append(r)
    

    # Set up figure
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,6))
    im = ax1.imshow(np.zeros((size, size)), cmap='bwr', origin='lower', vmin=-2, vmax=2)
    detector_line = ax1.axvline(x=detector_x, color='black', linestyle='--', linewidth=2, label='Detector')
    # add a color bar to ax1
    plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04)
    ax1.set_title("Wave Interference (Bird's-eye View)")
    ax1.legend()
    
    im2 = ax2.imshow(np.zeros((detector_height, size)), origin='lower', aspect='auto', cmap='bwr', vmin=0, vmax=1)
    ax2.set_title("Detector Intensity (2D)")
    for pos in slit_positions:
        ax1.hlines(y=pos, xmin=0, xmax=10, color='black', linewidth=5)

    def update(frame):
        t = frame / 5.0
        amplitude = np.zeros((size, size))
        for r in r_list:
            amplitude += np.sin(2 * np.pi * r / wavelength - 2*np.pi*t)
        im.set_array(amplitude)
        
        # Detector sees intensity along the column at detector_x
        intensity = amplitude[:, detector_x]**2
        intensity /= intensity.max()  # normalize for visibility
        
        # Convert to 2D by repeating the row
        detector_2d = np.tile(intensity, (detector_height,1))
        im2.set_array(detector_2d.T)
        
        return [im, im2, detector_line]

    anim = FuncAnimation(fig, update, frames=100, interval=50, blit=True)
    plt.show()

interact(
    diffraction_playground,
    detector_x=IntSlider(value=100, min=10, max=500, step=5, description='Detector position'),
    num_slits=IntSlider(value=2, min=1, max=50, step=1, description='Slits'),
    slit_width=IntSlider(value=3, min=1, max=50, step=1, description='Slit Width'),
    slit_sep=IntSlider(value=20, min=5, max=50, step=1, description='Slit Sep'),
    wavelength=FloatSlider(value=5.0, min=1.0, max=20.0, step=0.5, description='Wavelength'),
    random_spacing=Checkbox(value=False, description='Random Spacing')
)



interactive(children=(IntSlider(value=100, description='Detector position', max=500, min=10, step=5), IntSlide…

<function __main__.diffraction_playground(detector_x=100, num_slits=2, slit_width=3, slit_sep=20, wavelength=5.0, random_spacing=False)>